# Преобразование текста в эмбединги

*Использование моделей из sentence_transformers*

In [4]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Загружаем модель для создания embedding-векторов
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def cosine_similarity(text1, text2):
    # Преобразуем текст в embedding-векторы
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    
    # Вычисляем косинусное сходство
    cosine_sim = util.pytorch_cos_sim(embedding1, embedding2)
    
    return cosine_sim.item()  # Возвращаем значение сходства как float

# Пример данных
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение продает бургеры"

# Вычисляем косинусное сходство
similarity_score = cosine_similarity(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")


/home/user/Coding/easydocs/model-prototype/model-analyse/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Cosine Similarity: 0.2011


In [9]:
# Пример данных
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение использует MacOS и Windows"

# Вычисляем косинусное сходство
similarity_score = cosine_similarity(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")

Cosine Similarity: 0.6362


*Использование моделей Bert*

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Загружаем токенайзер и модель RuBERT
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

def get_embedding(text):
    # Токенизация текста и преобразование в тензоры
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Получаем выход модели
    with torch.no_grad():
        outputs = model(**inputs)
    # Используем эмбеддинг CLS-токена (первого токена) как представление текста
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu()
    return embedding

def cosine_similarity_rubert(text1, text2):
    # Получаем эмбеддинги для текста 1 и текста 2
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    # Вычисляем косинусное сходство
    cosine_sim = 1 - cosine(embedding1, embedding2)
    
    return cosine_sim

# Пример данных
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение поддерживает Windows и macOS."

# Вычисляем косинусное сходство
similarity_score = cosine_similarity_rubert(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum lengt

Cosine Similarity: 0.7674


In [12]:

# Пример данных
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение продает бургеры"

# Вычисляем косинусное сходство
similarity_score = cosine_similarity_rubert(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")

Cosine Similarity: 0.7714


*Использование тревиальных векторизаторов(не работает)*

In [8]:
import gensim.downloader

In [9]:
gensim.downloader.load('word2vec-ruscorpora-300')


[==================================================] 100.0% 198.8/198.8MB downloaded


In [16]:
import gensim.downloader as api
import numpy as np
from scipy.spatial.distance import cosine

# Загружаем предобученную модель Word2Vec
model = api.load('word2vec-ruscorpora-300')

def get_embedding(sentence):
    # Токенизируем предложение и получаем эмбеддинги для каждого слова
    words = sentence.split()
    embeddings = []
    
    for word in words:
        # Проверяем, есть ли слово в модели
        if word in model:
            embeddings.append(model[word])
    
    # Если не найдено ни одно слово, возвращаем нулевой вектор
    if not embeddings:
        return np.zeros(model.vector_size)
    
    # Усредняем эмбеддинги слов
    sentence_embedding = np.mean(embeddings, axis=0)
    return sentence_embedding

def cosine_similarity(text1, text2):
    # Получаем эмбеддинги для текстов
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    # Вычисляем косинусное сходство
    cosine_sim = 1 - cosine(embedding1, embedding2)
    
    return cosine_sim

# Пример использования
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение поддерживает Windows и macOS."

# Вычисляем косинусное сходство
similarity_score = cosine_similarity(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")


Cosine Similarity: nan


/home/user/Coding/easydocs/model-prototype/model-analyse/venv/lib/python3.12/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


*FastText - у меня оперативы не хватило*

In [1]:
import fasttext
import numpy as np
import fasttext.util

# fasttext.util.download_model('ru', if_exists='ignore')
# Загрузка предобученной модели
model = fasttext.load_model('cc.ru.300.bin')

# Получение среднего эмбеддинга для предложения
def get_sentence_embedding(text):
    words = text.split()
    word_embeddings = [model.get_word_vector(word) for word in words]
    return np.mean(word_embeddings, axis=0)


: 

In [4]:
import torch
import torch.nn.functional as F
from torchtext.vocab import GloVe
from scipy.spatial.distance import cosine
import numpy as np

# Загружаем предобученные GloVe эмбеддинги
glove = GloVe(name='6B', dim=100)  # Вы можете выбрать размер 50, 100, 200, 300

def get_embedding(sentence):
    # Токенизируем предложение и получаем эмбеддинги для каждого слова
    words = sentence.split()
    embeddings = []
    
    for word in words:
        # Проверяем, есть ли слово в GloVe
        if word in glove.stoi:
            embeddings.append(glove[word])
    
    # Если не найдено ни одно слово, возвращаем нулевой вектор
    if not embeddings:
        return torch.zeros(glove.dim)
    
    # Усредняем эмбеддинги слов
    sentence_embedding = torch.mean(torch.stack(embeddings), dim=0)
    return sentence_embedding

def cosine_similarity(text1, text2):
    # Получаем эмбеддинги для текстов
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    # Вычисляем косинусное сходство
    cosine_sim = 1 - cosine(embedding1.numpy(), embedding2.numpy())
    
    return cosine_sim

# Пример использования
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение поддерживает Windows и macOS."

# Вычисляем косинусное сходство
similarity_score = cosine_similarity(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")


/home/user/Coding/easydocs/model-prototype/model-analyse/venv/lib/python3.12/site-packages/torchtext/__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: /home/user/Coding/easydocs/model-prototype/model-analyse/venv/lib/python3.12/site-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs